<a href="https://colab.research.google.com/github/young-hwanlee/gnn_example/blob/main/gnn_basics_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/young-hwanlee/gnn_example.git

fatal: destination path 'gnn_example' already exists and is not an empty directory.


# **PyG Basics**

In [2]:
import torch

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
edge_index = torch.tensor([[0, 1, 1, 2],    # 출발 노드 인덱스
                                             [1, 0, 2, 1]],   # 도착 노드 인덱스
                          dtype=torch.long)

In [3]:
print(x)
print(edge_index)

tensor([[-1.],
        [ 0.],
        [ 1.]])
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])


In [4]:
from torch_geometric.data import Data

data = Data(x=x, edge_index=edge_index)
print(data)

Data(x=[3, 1], edge_index=[2, 4])


In [5]:
data.num_nodes

3

In [6]:
data.num_edges

4

In [7]:
data.has_isolated_nodes()

False

In [8]:
data.has_self_loops()

False

In [9]:
data.is_directed()

False

# **Dataset Example 1**

In [10]:
!pip install fsspec==2024.3.1

In [11]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES/', name='ENZYMES')
print(dataset)

ENZYMES(600)


In [12]:
len(dataset)

600

In [13]:
dataset.num_classes

6

In [14]:
dataset.num_node_features

3

In [15]:
data = dataset[0]
print(data)

Data(edge_index=[2, 168], x=[37, 3], y=[1])


In [16]:
data.is_undirected()

True

In [17]:
train_dataset = dataset[:540]
print(train_dataset)

ENZYMES(540)


In [18]:
test_dataset = dataset[540:]
print(test_dataset)

ENZYMES(60)


# **Dataset Example 2**

In [19]:
from torch_geometric.datasets import planetoid

dataset = planetoid.Planetoid(root='/tmp/Cora', name='Cora')
print(dataset)

Cora()


In [20]:
len(dataset)

1

In [21]:
dataset.num_classes

7

In [22]:
dataset.num_node_features

1433

In [23]:
data = dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [24]:
data.is_undirected()

True

In [25]:
data.train_mask.sum().item()

140

In [26]:
data.val_mask.sum().item()

500

In [27]:
data.test_mask.sum().item()

1000

# **Mini-Batch Loader**

In [28]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES/', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [29]:
print(loader)

In [30]:
for batch in loader:
  print(batch)
  print(batch.num_graphs)

DataBatch(edge_index=[2, 4630], x=[1171, 21], y=[32], batch=[1171], ptr=[33])
32
DataBatch(edge_index=[2, 4398], x=[1125, 21], y=[32], batch=[1125], ptr=[33])
32
DataBatch(edge_index=[2, 4186], x=[1040, 21], y=[32], batch=[1040], ptr=[33])
32
DataBatch(edge_index=[2, 3482], x=[885, 21], y=[32], batch=[885], ptr=[33])
32
DataBatch(edge_index=[2, 3838], x=[1064, 21], y=[32], batch=[1064], ptr=[33])
32
DataBatch(edge_index=[2, 4296], x=[1145, 21], y=[32], batch=[1145], ptr=[33])
32
DataBatch(edge_index=[2, 4112], x=[1013, 21], y=[32], batch=[1013], ptr=[33])
32
DataBatch(edge_index=[2, 3928], x=[1052, 21], y=[32], batch=[1052], ptr=[33])
32
DataBatch(edge_index=[2, 4294], x=[1140, 21], y=[32], batch=[1140], ptr=[33])
32
DataBatch(edge_index=[2, 3642], x=[961, 21], y=[32], batch=[961], ptr=[33])
32
DataBatch(edge_index=[2, 4008], x=[1109, 21], y=[32], batch=[1109], ptr=[33])
32
DataBatch(edge_index=[2, 3694], x=[976, 21], y=[32], batch=[976], ptr=[33])
32
DataBatch(edge_index=[2, 4026], x=

# **Data Transform**

In [31]:
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'], force_reload=True)
print(dataset[0])

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/dataset.py:239: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, pass `force_reload=True` explicitly to reload the dataset.
  warnings.warn(
Processing...


Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])


Done!


In [32]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                   pre_transform=T.KNNGraph(k=6),
                   transform=T.RandomJitter(0.01), force_reload=True)
print(dataset[0])

Processing...
Done!


Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1], edge_index=[2, 15108])
